In [358]:
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import pandas as pd
from datetime import datetime
import math 

from sql_functions import *    

import psycopg2   

import sql_functions as sf
from sql_functions import get_dataframe
import sqlalchemy

## Get all details

In [640]:
# get list of platform_ids from DBeaver first

# schema = 'capstone_jmrs'

# sql = f"""
# SELECT platform_id 
# FROM {schema}.rightmove_3
# """

#df = get_dataframe(sql)
#ids = df['platform_id'].tolist()[:10]
#------------------------------------------------------------
#------------------------------------------------------------


ids = df['platform_id'].tolist()

#ids = ['127325693', '127366049', '117388061', '127365830'] #for testing purposes only
df_details_complete = pd.DataFrame()
df_basics_complete = pd.DataFrame()

for id in ids:
    time.sleep(random.randint(2,6)/10)
    
    page = requests.get(f"https://www.rightmove.co.uk/properties/{id}#/?channel=RES_LET")
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # get details
    details = bs.find(
        'div', class_='_4hBezflLdgDMdFtURKTWh')
    details_lst = (detail.get_text() for detail in details)
    details_lst = [detail.strip() for detail in details_lst]

    #df_details_complete = pd.DataFrame()
    bedrooms, bathrooms, size, property_type = 'NA', 'NA', 'NA', 'NA' 
    row_dict = {'property_id':id, 'bedrooms': bedrooms, 'bathrooms': bathrooms, 'size': size, 'property_type': property_type}
    for detail in details_lst:
        if 'TYPE' in detail:
            row_dict['property_type'] = detail.removeprefix('PROPERTY TYPE')
        elif 'BEDROOMS' in detail:
            row_dict['bedrooms'] = detail[-1]
        elif 'BATHROOM' in detail:
            row_dict['bathrooms'] = detail[-1]
        elif 'SIZE' in detail:
            row_dict['size'] = re.search(r'\((.*?)\)', str(detail)).group(1).removesuffix(' sq. m.').replace(',', '')
    
    df_details = pd.DataFrame(row_dict, index=[0])    
    df_details_complete = pd.concat([df_details_complete, df_details], ignore_index=True)

    # get other basic details
    basics = bs.find_all(
        'div', class_='_2RnXSVJcWbWv4IpBC1Sng6')

    basics_lst = (basic.get_text() for basic in basics)
    basics_lst = [basic.strip().split(': ') for basic in basics_lst]    #'split'
    basics_lst

    # convert details_lst to dict
    itemDict = {item[0]: item[1] for item in basics_lst}
    itemDict['platform_id'] = id

    df_basics = pd.DataFrame(itemDict, index=[0])    
    df_basics_complete = pd.concat([df_basics_complete, df_basics], ignore_index=True)


# drop columns we don't need
df_basics_complete.drop(['Deposit', 'Min. Tenancy'], axis=1, inplace=True)

# pythonise column names 
df_basics_complete.columns.values[0:3] = ['available_from', 'let_type', 'furnished']

# concatenate both detail DFs
new_df = pd.concat([df_details_complete, df_basics_complete], axis=1)



# import the data frame to DBeaver
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = 'rightmove_details'

# import the table to sql
if engine!=None:
    try:
        new_df.to_sql(name=table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None


The rightmove_details table was imported successfully.


### end of working code

In [ ]:
# # doesnt take too long
# itemDict = {item[0]: item[1] for item in basics_lst}
# itemDict['property_id'] = id

# itemDict

In [408]:
# SIZE from detail page - Works, but what if there is no size?

sizes = bs_2.find(
    'p', class_='_3vyydJK3KMwn7-s2BEXJAf')

sizes_lst = (detail.get_text() for detail in sizes)
sizes_lst = [detail.strip() for detail in sizes_lst]

# convert string
# df['size'] = df['size'].str.removesuffix('sq. m.)').str.replace('(', '').str.replace(',', '')
sizes_lst

['(32 sq. m.)']

In [485]:
details = bs_2.find(
    'div', class_='_4hBezflLdgDMdFtURKTWh')
details_lst = (detail.get_text() for detail in details)
details_lst = [detail.strip() for detail in details_lst]
details_lst

['PROPERTY TYPEApartment', 'BATHROOMS×1']

In [466]:
## Would work, but breaks as soon as there is no size (maybe convert to dictionary first?)
page_2 = requests.get("https://www.rightmove.co.uk/properties/127325693#/?channel=RES_LET")
html_2 = page_2.content
bs_2 = BeautifulSoup(html_2, 'html.parser')


# TYPE  -- BEDROOMS -- BATHROOMS -- SIZE
details = bs_2.find(
    'div', class_='_4hBezflLdgDMdFtURKTWh')
details_lst = (detail.get_text() for detail in details)
details_lst = [detail.strip() for detail in details_lst]
details_lst

# Format entries
# details_lst[0] = str(details_lst[0]).replace('PROPERTY TYPE', '')
# details_lst[1] = str(details_lst[1]).replace('BEDROOMS×', '')
# details_lst[2] = str(details_lst[2]).replace('BATHROOMS×', '')
# #details_lst[3] = re.search(r'\((.*?)\)', str(details_lst[3])).group(1).removesuffix(' sq. m.').replace(',', '')

details_lst

['PROPERTY TYPEApartment', 'BATHROOMS×1']

## OVERVIEW PAGE

In [ ]:
# BEEDROOMS - WORKS!
bedrooms = [title.text.split('bedroom')[0].strip() for title in bs.findAll('h2', {'class': 'propertyCard-title'})]
bedrooms

In [ ]:
# TYPE - Working so far!
types_lst = [title.text.strip().split()[-1] for title in bs.findAll('h2', {'class': 'propertyCard-title'})]
types_lst

In [257]:
# BATHROOMS NOT WORKING AT ALL

bathrooms = bs.find_all(class_= "propertyCard-features")
bathrooms_lst = [bathroom.get_text() for bathroom in bathrooms]
bathrooms_lst = list(filter(None, bathrooms_lst)) #delete any empty strings from list
bathrooms_lst
# # format bathrooms
# bathrooms_lst = [bathroom.split('bathroom')[0].strip() for bathroom in bathrooms_lst]
# bathrooms_lst

[]

In [639]:
# Date added or reduced
dates_edited = [date.text for date in bs.findAll('span', {'class': 'propertyCard-branchSummary-addedOrReduced'})]
dates_edited

## possibly chec/update price in DB if value='reduced...'

[]

## Detail Code from Julia

In [359]:
from sql_functions import get_dataframe

In [356]:
schema = 'capstone_jmrs'

In [ ]:
sql = f"""
SELECT * 
FROM {schema}.rightmove_3
"""

df = get_dataframe(sql)
df

In [ ]:
df_city = df.query('neighbourhood == "City of London"')
df_city.head(15)